# Leyendo y pre-procesando los datos

In [150]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [151]:
#Imstalamos la libreria pycountry
!pip install pycountry==1.20

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [152]:
#Importamos las librerias
import numpy as np
import pandas as pd
from datetime import datetime

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.offline as py
import pycountry



%config InlineBackend.figure_format = 'retina'

plt.rcParams['figure.figsize'] = 8, 5

import warnings
warnings.filterwarnings('ignore')
import os

In [153]:
#Leemos el documento con formato csv
path='/content/drive/MyDrive/Forbes Richest Atheletes (Forbes Richest Athletes 1990-2020).csv'
df = pd.read_csv(path)

# Data de los atletas más ricos según forbes

In [154]:
#Lo mostramos en pantalla
df

,S.NO,Name,Nationality,Current Rank,Previous Year Rank,Sport,Year,earnings ($ million)
0,1,Mike Tyson,USA,1,NaN,boxing,1990,28.6
1,2,Buster Douglas,USA,2,NaN,boxing,1990,26.0
2,3,Sugar Ray Leonard,USA,3,NaN,boxing,1990,13.0
3,4,Ayrton Senna,Brazil,4,NaN,auto racing,1990,10.0
4,5,Alain Prost,France,5,NaN,auto racing,1990,9.0
...,...,...,...,...,...,...,...,...
296,297,Stephen Curry,USA,6,9,Basketball,2020,74.4
297,298,Kevin Durant,USA,7,10,Basketball,2020,63.9
298,299,Tiger Woods,USA,8,11,Golf,2020,62.3
299,300,Kirk Cousins,USA,9,>100,American Football,2020,60.5


In [155]:
#Vemos la infoemación del dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   S.NO                  301 non-null    int64  
 1   Name                  301 non-null    object 
 2   Nationality           301 non-null    object 
 3   Current Rank          301 non-null    int64  
 4   Previous Year Rank    277 non-null    object 
 5   Sport                 301 non-null    object 
 6   Year                  301 non-null    int64  
 7   earnings ($ million)  301 non-null    float64
dtypes: float64(1), int64(3), object(4)
memory usage: 18.9+ KB


In [156]:
#Se crea una copia del dataframe df nombrada df1
df1 = df.copy()
#Eliminamos la columna S.NO
df1.drop('S.NO', axis = 1, inplace = True)


In [157]:
#Convertir la columna Year de int64 a formato datetime64
df1['Year'] = df1['Year'].apply(pd.to_datetime,format = '%Y')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Name                  301 non-null    object        
 1   Nationality           301 non-null    object        
 2   Current Rank          301 non-null    int64         
 3   Previous Year Rank    277 non-null    object        
 4   Sport                 301 non-null    object        
 5   Year                  301 non-null    datetime64[ns]
 6   earnings ($ million)  301 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 16.6+ KB


In [158]:
df1.head(2)

,Name,Nationality,Current Rank,Previous Year Rank,Sport,Year,earnings ($ million)
0,Mike Tyson,USA,1,NaN,boxing,1990-01-01,28.6
1,Buster Douglas,USA,2,NaN,boxing,1990-01-01,26.0


In [159]:
#Cambiamos el nombre y agarramos de Year el formato de años que contiene esa columna
df1['year'] = pd.DatetimeIndex(df1['Year']).year
#Remplazamos los indices por la columna year
df1.set_index('year', inplace=True)
#Eliminamos la columna Year
df1.drop('Year',axis=1,inplace=True)

In [160]:
#De la columna Sport utilizamos upper para hacer que las letras se vuelvan mayusculas 
df1['Sport'] = df1['Sport'].str.upper()
df1.head(2)

,Name,Nationality,Current Rank,Previous Year Rank,Sport,earnings ($ million)
year,,,,,,
1990,Mike Tyson,USA,1,NaN,BOXING,28.6
1990,Buster Douglas,USA,2,NaN,BOXING,26.0


# Los atletas mejor pagados en 2020

In [161]:
#
data_2020 = df1[df1.index == 2020]
data_2020.head(2)


,Name,Nationality,Current Rank,Previous Year Rank,Sport,earnings ($ million)
year,,,,,,
2020,Roger Federer,Switzerland,1,5,TENNIS,106.3
2020,Cristiano Ronaldo,Portugal,2,2,SOCCER,105.0


In [162]:
trace = go.Bar(
                    x = data_2020["earnings ($ million)"],
                    y = data_2020['Name'] ,
                    orientation='h',
                    marker = dict(color='pink',
                                 line=dict(color='black',width=1)),
                    )
data = [trace]


In [163]:
layout = go.Layout(barmode = "group",title="Los atletas mayor pagados en el mundo en 2020",width=800, height=500, 
                       xaxis= dict(title='Ranking de las ganascias de los atletas en 2020'),
                       yaxis=dict(autorange="reversed"),
                       showlegend=False)

In [164]:
fig = go.Figure(data = data, layout = layout)
iplot(fig)

# Análisis de los atletas mejor pagados entre 1990 y 2020.

In [165]:
#Filtran a los deportistas con por año y rank = 1 
Top_paid_each_year = df1[df1['Current Rank'] == 1].sort_values(by='year',ascending=False)
Top_paid_each_year

,Name,Nationality,Current Rank,Previous Year Rank,Sport,earnings ($ million)
year,,,,,,
2020,Roger Federer,Switzerland,1,5,TENNIS,106.3
2019,Lionel Messi,Argentina,1,2,SOCCER,127.0
2018,Floyd Mayweather,USA,1,>100,BOXING,285.0
2017,Cristiano Ronaldo,Portugal,1,1,SOCCER,93.0
2016,Cristiano Ronaldo,Portugal,1,3,SOCCER,88.0
2015,Floyd Mayweather,USA,1,1,BOXING,300.0
2014,Floyd Mayweather,USA,1,14,BOXING,105.0
2013,Tiger Woods,USA,1,3,GOLF,78.1
2012,Floyd Mayweather,USA,1,?,BOXING,85.0


In [166]:
z = Top_paid_each_year[['Name','Sport','Nationality','earnings ($ million)']]
#Utilizando background_gradient 
z.style.background_gradient(cmap='Reds') 

,Name,Sport,Nationality,earnings ($ million)
year,,,,
2020,Roger Federer,TENNIS,Switzerland,106.300000
2019,Lionel Messi,SOCCER,Argentina,127.000000
2018,Floyd Mayweather,BOXING,USA,285.000000
2017,Cristiano Ronaldo,SOCCER,Portugal,93.000000
2016,Cristiano Ronaldo,SOCCER,Portugal,88.000000
2015,Floyd Mayweather,BOXING,USA,300.000000
2014,Floyd Mayweather,BOXING,USA,105.000000
2013,Tiger Woods,GOLF,USA,78.100000
2012,Floyd Mayweather,BOXING,USA,85.000000


## Atletas con mayor ingreso y máximo número de apariciones en la lista Forbes

In [167]:
#Se filtran las veces que aparecieron en cada año como los mejores
counts_top = Top_paid_each_year['Name'].value_counts().to_frame()
counts_top

,Name
Tiger Woods,11
Michael Jordan,6
Floyd Mayweather,4
Cristiano Ronaldo,2
Michael Schumacher,2
Mike Tyson,2
Roger Federer,1
Lionel Messi,1
Evander Holyfield,1


In [168]:
trace = go.Bar(
                    y = counts_top.index, #Da los valores de la primer columna
                    x = counts_top['Name'] ,
                    orientation='h',
                    marker = dict(color='pink',
                                 line=dict(color='black',width=1)),
                    )
data = [trace]
layout = go.Layout(barmode = "group",title='Numero de veces de atletas que tuvieron las mayores ganancias',width=800, height=500, 
                       xaxis= dict(title='Numero de veces en el top'),
                       yaxis=dict(autorange="reversed"),
                       showlegend=False)
fig = go.Figure(data = data, layout = layout)
iplot(fig)

In [169]:
# Top Paid Athlete for Each Year
total_earnings = Top_paid_each_year.groupby('Name')['earnings ($ million)'].sum().to_frame().reset_index()
top_ranks = Top_paid_each_year['Name'].value_counts().to_frame().reset_index()
top_ranks.rename(columns={'index':'Name',
                          'Name':'Rank_counts'}, inplace=True)
df_compare = total_earnings.merge(top_ranks, on='Name')
df_compare.sort_values(by='earnings ($ million)', ascending=False)

,Name,earnings ($ million),Rank_counts
8,Tiger Woods,987.4,11
2,Floyd Mayweather,775.0,4
4,Michael Jordan,293.1,6
0,Cristiano Ronaldo,181.0,2
3,Lionel Messi,127.0,1
5,Michael Schumacher,108.0,2
7,Roger Federer,106.3,1
6,Mike Tyson,103.6,2
1,Evander Holyfield,60.5,1


# Pais que produce los máximos generadores de ingreso en deporte.

In [170]:
#Filtramos la nacionalidad de los mejores pagados en cada año
counts_top = Top_paid_each_year['Nationality'].value_counts().to_frame()

trace = go.Bar(
                    x = counts_top.index,
                    y = counts_top['Nationality'] ,
                    orientation='v',
                    marker = dict(color='pink',
                                 line=dict(color='black',width=1)),
                    )
data = [trace]
layout = go.Layout(barmode = "group",title='Paises que producen las mayores ganancias en los deportes',width=800, height=500, 
                       xaxis= dict(title='Numero de veces en el top del rank'),
                       #yaxis=dict(autorange="reversed"),
                       showlegend=False)
fig = go.Figure(data = data, layout = layout)
iplot(fig)

In [171]:
trace = go.Scatter(
                    x = Top_paid_each_year.index,
                    y = Top_paid_each_year['earnings ($ million)'] ,
                    orientation='v',
                    marker = dict(color='red',
                                 line=dict(color='royalblue',width=2)),
                    )
data = [trace]
layout = go.Layout(title='¿Cuánto se le paga al atleta que estuvo en el top cada año? ',width=800, height=500, 
                       xaxis= dict(title='Años'),
                       yaxis=dict(title="Ganancias en US Dollars(million)"),
                       showlegend=False)
fig = go.Figure(data = data, layout = layout)
iplot(fig)

#¿Cuánto ingresan los mejores atletas cada año?

In [172]:
df['Sport'] = df['Sport'].str.upper()

#Cuenta la cantidad de deportes que se repiten por jugadores mejor pagados y lo regresa en un dataframe
max_sport = df['Sport'].value_counts().to_frame()
max_sport.head()

,Sport
BASKETBALL,81
BOXING,46
GOLF,44
SOCCER,33
TENNIS,23


In [173]:
trace = go.Bar(
                    y = max_sport.index,
                    x = max_sport['Sport'] ,
                    orientation='h',
                    marker = dict(color='pink',
                                 line=dict(color='black',width=1)),
                    )
data = [trace]
layout = go.Layout(barmode = "group",title='Deportes que dominan en ganancias',width=800, height=500, 
                       xaxis= dict(title='Numero de veces que aparece cada deporte'),
                       yaxis=dict(autorange="reversed"),
                       showlegend=False)
fig = go.Figure(data = data, layout = layout)
iplot(fig)

In [174]:
max_sport = df['Nationality'].value_counts().to_frame()
max_sport.head()

,Nationality
USA,206
UK,13
Germany,13
Switzerland,12
Portugal,10


In [175]:
trace = go.Bar(
                    y = max_sport.index,
                    x = max_sport['Nationality'] ,
                    orientation='h',
                    marker = dict(color='pink',
                                 line=dict(color='black',width=1)),
                    )
data = [trace]
layout = go.Layout(barmode = "group",title='Paises los cuales dominan en las ganancias deportivas ',width=800, height=500, 
                       xaxis= dict(title='Numero de veces que aparece el pais'),
                       yaxis=dict(autorange="reversed"),
                       showlegend=False)
fig = go.Figure(data = data, layout = layout)
iplot(fig)

# Atletas que aparecen más frecuentemente en la lista

In [176]:
# [:5] define cuantas filas va almacenar 
s = df['Name'].value_counts().to_frame()[:5]
s

,Name
Tiger Woods,19
Michael Jordan,19
Kobe Bryant,14
LeBron James,13
Michael Schumacher,13


In [177]:
s.style.background_gradient(cmap='Reds')

,Name
Tiger Woods,19
Michael Jordan,19
Kobe Bryant,14
LeBron James,13
Michael Schumacher,13


# ¿Dónde se encuentran las mujeres?

In [178]:
names = df['Name'].value_counts().to_frame()
names

,Name
Tiger Woods,19
Michael Jordan,19
Kobe Bryant,14
LeBron James,13
Michael Schumacher,13
...,...
Tom Brady,1
Michael Moorer,1
Dale Earnhardt Jr.,1
Greg Norman,1


In [179]:
#Nos da el nombre de los que solo han sido una vez los mejores pagados
names[names['Name']==1].index

Index(['Matthew Stafford', 'Aaron Rodgers', 'Rafael Nadal', 'Kirk Cousins',
       'Aaron Rogers', 'Novak Djokovic', 'Jordan Spieth', 'Cam Newton',
       'Canelo Alvarez', 'Andrew Luck', 'Rory McIlroy', 'Drew Brees',
       'James Harden', 'Lewis Hamilton', 'Russell Wilson', 'Conor McGregor',
       'Deion Sanders', 'Donovan "Razor" Ruddock', 'Terrell Suggs',
       'Eli Manning', 'Emmit Smith', 'Dennis Rodman', 'Gerhard Berger',
       'Joe Sakic', 'Cecil Fielder', 'Sergei Federov', 'Gary Sheffield',
       'Jeff Gordon', 'Buster Douglas', 'Monica Seles', 'Michael Vick',
       'Lance Armstrong', 'Muhammad Ali', 'Tom Brady', 'Michael Moorer',
       'Dale Earnhardt Jr.', 'Greg Norman', 'Carson Wentz'],
      dtype='object')

In [180]:
#le indicamos que solo queremos ver a Monica Seles
monica = df[df['Name'] == 'Monica Seles']
monica

,S.NO,Name,Nationality,Current Rank,Previous Year Rank,Sport,Year,earnings ($ million)
29,30,Monica Seles,USA,10,12,TENNIS,1992,8.5


In [181]:
monica.style.set_properties(**{'background-color': 'pink',
                            'color': 'black',
                            'border-color': 'black'})

,S.NO,Name,Nationality,Current Rank,Previous Year Rank,Sport,Year,earnings ($ million)
29,30,Monica Seles,USA,10,12,TENNIS,1992,8.500000


# Analizando a los 3 mejores pagados de todos los tiempos.

In [182]:
top_earners_alltime = pd.pivot_table(df, index='Name',values="earnings ($ million)", aggfunc='sum')
top_earners_alltime

,earnings ($ million)
Name,
Aaron Rodgers,49.0
Aaron Rogers,89.3
Alain Prost,36.0
Alex Rodriguez,116.2
Andre Agassi,105.8
...,...
Terrell Suggs,38.3
Tiger Woods,1373.8
Tom Brady,29.0


In [183]:
top3_earners_all = top_earners_alltime.sort_values(by="earnings ($ million)",ascending=False)[:3]
top3_earners_all

,earnings ($ million)
Name,
Tiger Woods,1373.8
LeBron James,844.8
Floyd Mayweather,840.0


In [184]:
top3_earners_all.style.background_gradient(cmap='Reds')

,earnings ($ million)
Name,
Tiger Woods,1373.800000
LeBron James,844.800000
Floyd Mayweather,840.000000


In [185]:
rango_10_20 = df1[df1.index >= 2010]
rango_10_20.head(5)

,Name,Nationality,Current Rank,Previous Year Rank,Sport,earnings ($ million)
year,,,,,,
2010,Tiger Woods,USA,1,1,GOLF,105.0
2010,Floyd Mayweather,USA,2,>10,BOXING,65.0
2010,Kobe Bryant,USA,3,2,BASKETBALL,48.0
2010,Phil Mickelson,USA,4,6,GOLF,46.0
2010,David Beckham,UK,5,5,SOCCER,43.7


In [186]:
rango_10_20[rango_10_20['Current Rank'] <=2].head(2)

,Name,Nationality,Current Rank,Previous Year Rank,Sport,earnings ($ million)
year,,,,,,
2010,Tiger Woods,USA,1,1,GOLF,105.0
2010,Floyd Mayweather,USA,2,>10,BOXING,65.0


In [187]:
datos=df1[(df1.index<=2020) & (df1.index>=2010)]
datos.head(2)

,Name,Nationality,Current Rank,Previous Year Rank,Sport,earnings ($ million)
year,,,,,,
2010,Tiger Woods,USA,1,1,GOLF,105.0
2010,Floyd Mayweather,USA,2,>10,BOXING,65.0


In [188]:
datos['Previous Year Rank'].unique()
datos['Current Rank'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [189]:
datos['Previous Year Rank'] = datos['Previous Year Rank'].str.replace('>10', '10')
datos['Previous Year Rank'] = datos['Previous Year Rank'].str.replace('>20', '20')
datos['Previous Year Rank'] = datos['Previous Year Rank'].str.replace('>14', '14')
datos['Previous Year Rank'] = datos['Previous Year Rank'].str.replace('?', '100')
datos['Previous Year Rank'] = datos['Previous Year Rank'].str.replace('100100', '100')

In [190]:
datos=datos.fillna(100)

In [191]:
datos['Previous Year Rank'].unique()

array(['1', '10', '2', '6', '5', '3', '7', '4', '13', '12', '20', '100',
       '24', '21', '8', '9', '11', '14', 100, '17', '18', '15'],
      dtype=object)

In [192]:
#subieron o bajaron
datos['Posiciones subidas']=datos['Previous Year Rank'].astype(int)-datos['Current Rank'].astype(int)



# Jugadores que subieron al menos dos lugares 

In [193]:
J2=pd.DataFrame(datos[datos['Posiciones subidas']>=2]['Name'].unique())
J2=J2.rename(columns={0:'Jugadores que subieron al menos dos lugares'})
J2

,Jugadores que subieron al menos dos lugares
0,Floyd Mayweather
1,Phil Mickelson
2,Roger Federer
3,LeBron James
4,Cristiano Ronaldo
5,Alex Rodriguez
6,Michael Schumacher
7,Lionel Messi
8,Manny Pacquiao
9,Tiger Woods
